<a href="https://colab.research.google.com/github/Czedros/CSE352-Machine-Learning-Assignments/blob/main/Knowledge-based/z3logic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

Name: Kay Zhang

I understand that my submission needs to be my own work: [add initials]

I understand that ChatGPT / Copilot / other AI tools are not allowed: [add initials]

Total Points: **75**

Complete this notebook and submit it (save/print as pdf). Make sure all output is correct in the pdf before submitting (it sometimes gets cut off). The notebook needs to be a complete project report with your implementation, documentation including a short discussion of how your implementation works and your design choices, and experimental results (e.g., tables and charts with simulation results) with a short discussion of what they mean. Use the provided notebook cells and insert additional code and markdown cells as needed.

## Z3 and Z3py resources

For this problem you will be solving constraint satisfaction and logic problems using the Z3 SMT solver via the z3py python library.

The demo code using z3py we went over in class is here: [link](https://drive.google.com/drive/folders/16HQXiwdcaman1IpC7H6vSJ71QQQ-aL2z?usp=sharing)

Online help for z3py is here: [link](https://ericpony.github.io/z3py-tutorial/guide-examples.htm)

## Installation

We first install to get the z3solver library using pip and test that works.

In [16]:
!pip install z3-solver

In [17]:
# Run the first example from the z3py guide as a test: https://ericpony.github.io/z3py-tutorial/guide-examples.htm
from z3 import *

x = Int('x')
y = Int('y')
solve(x > 2, y < 10, x + 2*y == 7)

[y = 0, x = 7]


# Sudoku as a constraint satisfaction problem (CSP)

Sudoku is a popular number-placement puzzle that originated in France in the end of the 19th century.  Modern Sudoku was likely invented by Howard Garns from Connersville, Indiana and was first published in 1979 under the name *Number Place*. The objective of the puzzle is to place numbers 1-9 on a 9$\times$9 grid, such that each number occurs only once in every row, every column, and every of the nine 3$\times$3 sub-grids that compose the main grid.
Sudoku puzzles are grids that have been partially occupied with numbers. The task is then to occupy the remaining fields in such a way that the constraints on rows, columns, and sub-grids are satisfied.
For more information about Sudoku refer to its Wikipedia page at http://en.wikipedia.org/wiki/Sudoku.

This problem has two parts. In the first part, you will write the boolean constraints in mathematical notation for solving a Sudoku puzzle. In the second part, you will write code and invoke the Z3 solver to solve the Sudoku instance and answer various questions about the solution.

## Part 1 (Constraints)
In text, define and write constraints over **boolean** variables corresponding to each number being in each cell. For example, you may use $X_{i,j}^k$ as teh variable that is true if and only if the number $k$ is in row $i$ and column $j$ (the variable is true if $k$ is in cell $(i, j)$ and false otherwise). Now write the following boolean constaints over these variables:

* Write the boolean formula for the constraints that the number 5 can occur at most once in the first row. **(3 point)**
* Write the boolean formula for the constraints that the number 6 can occur at most once in the first column. **(3 point)**
* Write the boolean formula for the constraints that the number 9 can occur at most once in the top left 3$\times$3 sub-grid. **(4 point)**

Solution:

Let $X_{i,j}^k$ be a boolean variable such that

$X_{i,j}^k$ = true if cell(i,j) contains number k

and

$X_{i,j}^k$ = false otherwise

1.
$\bigwedge_{1 \leq j < l \leq 9} \neg (X_{1,j}^5 \land X_{1,l}^5)$

2.
$\bigwedge_{1 \leq i < m \leq 9} \neg (X_{i,1}^6 \land X_{m,1}^6)$

3.$\bigwedge_{(i_1, j_1) < (i_2, j_2)} \neg \left( X_{i_1, j_1}^9 \land X_{i_2, j_2}^9 \right)$


## Part 2 (Coding)

Encode the above constraints and all the other ones as a CSP using z3py and solve the Sudoku instance given in the figure below. **Use only boolean variables** and do not use the **Distinct** function.

* Provide the code to solve that solves the given problem instance using z3py and only boolean variables. The code should output a reasonable visualization of the solution, for example printed in text. **(10 points)**

<img src="https://raw.githubusercontent.com/stanleybak/CS7320-AI/master/Knowledge-based/sudoku1.jpg" width="400"/>



In [18]:
# z3py code to solve above Sudoku goes here. Do not use Int(*), only Bool(*)
from z3 import *

solver = Solver()

X = [[[Bool(f"X_{i}_{j}_{k}") for k in range(9)] for j in range(9)] for i in range(9)]


# 1. Each cell contains exactly one number
for i in range(9):
    for j in range(9):
        solver.add(Or(X[i][j]))
        for k1 in range(9):
            for k2 in range(k1 + 1, 9):
                solver.add(Or(Not(X[i][j][k1]), Not(X[i][j][k2])))

# 2. Each number appears at most once in each row
for i in range(9):
    for k in range(9):
        for j1 in range(9):
            for j2 in range(j1 + 1, 9):
                solver.add(Or(Not(X[i][j1][k]), Not(X[i][j2][k])))

# 3. Each number appears at most once in each column
for j in range(9):
    for k in range(9):
        for i1 in range(9):
            for i2 in range(i1 + 1, 9):
                solver.add(Or(Not(X[i1][j][k]), Not(X[i2][j][k])))

# 4. Each number appears at most once in each 3x3 block
for bi in range(3):
    for bj in range(3):
        for k in range(9):
            block_cells = [(bi*3 + di, bj*3 + dj) for di in range(3) for dj in range(3)]
            for idx1 in range(9):
                for idx2 in range(idx1 + 1, 9):
                    i1, j1 = block_cells[idx1]
                    i2, j2 = block_cells[idx2]
                    solver.add(Or(Not(X[i1][j1][k]), Not(X[i2][j2][k])))


puzzle = [
    [0, 1, 0, 4, 0, 2, 0, 5, 0],
    [5, 0, 0, 0, 0, 0, 0, 0, 6],
    [0, 0, 0, 3, 0, 1, 0, 0, 0],
    [7, 0, 5, 0, 0, 0, 4, 0, 8],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [2, 0, 8, 0, 0, 0, 5, 0, 9],
    [0, 0, 0, 9, 0, 6, 0, 0, 0],
    [6, 0, 0, 0, 0, 0, 0, 0, 2],
    [0, 7, 0, 1, 0, 3, 0, 4, 0],
]

for i in range(9):
    for j in range(9):
        if puzzle[i][j] != 0:
            k = puzzle[i][j] - 1
            solver.add(X[i][j][k])

def print_grid(model, X):
    for i in range(9):
        if i % 3 == 0 and i != 0:
            print("-" * 21)
        row = ""
        for j in range(9):
            if j % 3 == 0 and j != 0:
                row += "| "
            for k in range(9):
                if is_true(model.evaluate(X[i][j][k])):
                    row += str(k + 1) + " "
        print(row.strip())

if solver.check() == sat:
    model = solver.model()
    print_grid(model, X)
else:
    print("No solution found.")



3 1 9 | 4 6 2 | 8 5 7
5 2 4 | 7 9 8 | 1 3 6
8 6 7 | 3 5 1 | 9 2 4
---------------------
7 3 5 | 2 1 9 | 4 6 8
1 9 6 | 8 4 5 | 2 7 3
2 4 8 | 6 3 7 | 5 1 9
---------------------
4 5 3 | 9 2 6 | 7 8 1
6 8 1 | 5 7 4 | 3 9 2
9 7 2 | 1 8 3 | 6 4 5


### Uniqueness
Is your solution unique? Prove it with a Z3py solver or provide a second solution. **(5 points)**

In [19]:
if solver.check() == sat:
    model1 = solver.model()

    # Store current solution in a list
    solution = [[-1 for _ in range(9)] for _ in range(9)]
    for i in range(9):
        for j in range(9):
            for k in range(9):
                if is_true(model1.evaluate(X[i][j][k])):
                    solution[i][j] = k

    # Create a new constraint: there exists at least one cell different
    diff_constraint = []
    for i in range(9):
        for j in range(9):
            current_k = solution[i][j]
            diff_constraint.append(Not(X[i][j][current_k]))

    # Push solver state and add the negation of current solution
    solver.push()
    solver.add(Or(diff_constraint))
    result = solver.check()

    if result == unsat:
        print("The solution is Unique.")
    else:
        print("There is another solution.")
        model2 = solver.model()
        print_grid(model2, X)

    solver.pop()
else:
    print("No solution found.")

The solution is Unique.


### More Uniqueness
If you delete the 1 in the top left box of the Sudoku problem above, how many unique solutions are there? Hint: should be fewer than 5000. **(5 points)**

In [20]:
from z3 import *

def count_all_solutions(puzzle):
    solver = Solver()
    X = [[[Bool(f"X_{i}_{j}_{k}") for k in range(9)] for j in range(9)] for i in range(9)]

    # Sudoku constraints (same as before)
    for i in range(9):
        for j in range(9):
            solver.add(Or(X[i][j]))
            for k1 in range(9):
                for k2 in range(k1 + 1, 9):
                    solver.add(Or(Not(X[i][j][k1]), Not(X[i][j][k2])))

    for i in range(9):
        for k in range(9):
            for j1 in range(9):
                for j2 in range(j1 + 1, 9):
                    solver.add(Or(Not(X[i][j1][k]), Not(X[i][j2][k])))

    for j in range(9):
        for k in range(9):
            for i1 in range(9):
                for i2 in range(i1 + 1, 9):
                    solver.add(Or(Not(X[i1][j][k]), Not(X[i2][j][k])))

    for bi in range(3):
        for bj in range(3):
            for k in range(9):
                cells = [(bi*3 + di, bj*3 + dj) for di in range(3) for dj in range(3)]
                for a in range(len(cells)):
                    for b in range(a+1, len(cells)):
                        i1, j1 = cells[a]
                        i2, j2 = cells[b]
                        solver.add(Or(Not(X[i1][j1][k]), Not(X[i2][j2][k])))

    for i in range(9):
        for j in range(9):
            if puzzle[i][j] != 0:
                k = puzzle[i][j] - 1
                solver.add(X[i][j][k])

    # Enumerate all solutions
    count = 0
    seen = []
    while solver.check() == sat:
        model = solver.model()
        count += 1
        # Extract current solution
        current = []
        for i in range(9):
            for j in range(9):
                for k in range(9):
                    if is_true(model.evaluate(X[i][j][k])):
                        current.append(X[i][j][k])
                        break

        solver.add(Not(And(current)))

    print(f" Total solutions count: {count}")
    return count

puzzle = [
    [0, 0, 0, 4, 0, 2, 0, 5, 0],
    [5, 0, 0, 0, 0, 0, 0, 0, 6],
    [0, 0, 0, 3, 0, 1, 0, 0, 0],
    [7, 0, 5, 0, 0, 0, 4, 0, 8],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [2, 0, 8, 0, 0, 0, 5, 0, 9],
    [0, 0, 0, 9, 0, 6, 0, 0, 0],
    [6, 0, 0, 0, 0, 0, 0, 0, 2],
    [0, 7, 0, 1, 0, 3, 0, 4, 0],
]
count_all_solutions(puzzle)

 Total solutions count: 200


200

# Miracle Sudoku

Using z3py and only boolean variables, solve the Miracle Sudoku ([spoiler](https://www.youtube.com/watch?v=yKf9aUIxdb4)).
In this puzzle normal Sudoku rules apply in addition to the following rules:
* Any two cells separated by a knight's move from chess (moving forward two boxes and over one, in any direction) cannot contain the same digit (see image below).
* Any two cells separated by a king's move from chess (one box in any direction including diagonals) cannot contain the same digit.
* Any two orthogonally adjacent (up, down, left, or right) cells cannot contain consecutive digits.

### Knight's move constraint image
<img src="https://raw.githubusercontent.com/stanleybak/CS7320-AI/master/Knowledge-based/knights.jpg" width="400"/>

The knight's move constraint would mean the yellow boxes cannot contain a 1.


## Miracle Sudoku Starting Information
Provide the code to solve the puzzle given below. The code should output a reasonable visualization of the solution, for example using text. **(10 points)**

<img src="https://raw.githubusercontent.com/stanleybak/CS7320-AI/master/Knowledge-based/miracle.jpg" width="400"/>

In [21]:
from z3 import *

def print_grid(model, X):
    for i in range(9):
        if i % 3 == 0 and i != 0:
            print("-" * 21)
        row = ""
        for j in range(9):
            if j % 3 == 0 and j != 0:
                row += "| "
            for k in range(9):
                if is_true(model.evaluate(X[i][j][k])):
                    row += str(k + 1) + " "
                    break
        print(row.strip())

def solve_miracle_sudoku(puzzle):
    solver = Solver()
    X = [[[Bool(f"X_{i}_{j}_{k}") for k in range(9)] for j in range(9)] for i in range(9)]

    def in_bounds(i, j):
        return 0 <= i < 9 and 0 <= j < 9

    knight_moves = [(-2, -1), (-1, -2), (-2, 1), (-1, 2), (1, -2), (2, -1), (2, 1), (1, 2)]
    king_moves = [(-1, -1), (-1, 0), (-1, 1),
                  (0, -1),           (0, 1),
                  (1, -1),  (1, 0),  (1, 1)]
    orthogonal = [(-1, 0), (1, 0), (0, -1), (0, 1)]

    # Standard Sudoku constraints
    for i in range(9):
        for j in range(9):
            solver.add(Or(X[i][j]))
            for k1 in range(9):
                for k2 in range(k1 + 1, 9):
                    solver.add(Or(Not(X[i][j][k1]), Not(X[i][j][k2])))

    for k in range(9):
        for i in range(9):
            for j1 in range(9):
                for j2 in range(j1 + 1, 9):
                    solver.add(Or(Not(X[i][j1][k]), Not(X[i][j2][k])))
        for j in range(9):
            for i1 in range(9):
                for i2 in range(i1 + 1, 9):
                    solver.add(Or(Not(X[i1][j][k]), Not(X[i2][j][k])))

    for bi in range(3):
        for bj in range(3):
            for k in range(9):
                block = [(bi*3 + di, bj*3 + dj) for di in range(3) for dj in range(3)]
                for a in range(9):
                    for b in range(a + 1, 9):
                        i1, j1 = block[a]
                        i2, j2 = block[b]
                        solver.add(Or(Not(X[i1][j1][k]), Not(X[i2][j2][k])))

    # Miracle Sudoku constraints
    for i in range(9):
        for j in range(9):
            for dx, dy in knight_moves:
                ni, nj = i + dx, j + dy
                if in_bounds(ni, nj):
                    for k in range(9):
                        solver.add(Or(Not(X[i][j][k]), Not(X[ni][nj][k])))

            for dx, dy in king_moves:
                ni, nj = i + dx, j + dy
                if in_bounds(ni, nj):
                    for k in range(9):
                        solver.add(Or(Not(X[i][j][k]), Not(X[ni][nj][k])))

            for dx, dy in orthogonal:
                ni, nj = i + dx, j + dy
                if in_bounds(ni, nj):
                    for k in range(9):
                        if k > 0:
                            solver.add(Or(Not(X[i][j][k]), Not(X[ni][nj][k - 1])))
                        if k < 8:
                            solver.add(Or(Not(X[i][j][k]), Not(X[ni][nj][k + 1])))

    # Input puzzle values
    for i in range(9):
        for j in range(9):
            if puzzle[i][j] != 0:
                k = puzzle[i][j] - 1
                solver.add(X[i][j][k])

    if solver.check() == sat:
        model = solver.model()
        print_grid(model, X)
    else:
        print("No solution found.")

# "1" at (4,2), "2" at (4,6) — all other cells are empty
puzzle = [[0 for _ in range(9)] for _ in range(9)]
puzzle[4][2] = 1
puzzle[5][6] = 2

solve_miracle_sudoku(puzzle)


4 8 3 | 7 2 6 | 1 5 9
7 2 6 | 1 5 9 | 4 8 3
1 5 9 | 4 8 3 | 7 2 6
---------------------
8 3 7 | 2 6 1 | 5 9 4
2 6 1 | 5 9 4 | 8 3 7
5 9 4 | 8 3 7 | 2 6 1
---------------------
3 7 2 | 6 1 5 | 9 4 8
6 1 5 | 9 4 8 | 3 7 2
9 4 8 | 3 7 2 | 6 1 5


---
# Number Challenge in Z3py

Write Z3py code to solve the Number Challenge puzzle shown below and print the solution found. You can use the `Int` type to create integer variables in z3py. **(20 points)**

<img src="https://raw.githubusercontent.com/stanleybak/CS7320-AI/master/Knowledge-based/chal.jpg" width="300"/>

In [23]:
# Write your solution code for Number Challenge using z3py
from z3 import *

cells = [Int(f"c{i}") for i in range(8)]

s = Solver()

# Getting Possible Values
for c in cells:
    s.add(c >= 1, c <= 8)
s.add(Distinct(cells))

# Creating a Map of valid positions
positions = {
    0: (0, 1),
    1: (0, 2),
    2: (1, 0),
    3: (1, 1),
    4: (1, 2),
    5: (1, 3),
    6: (2, 1),
    7: (2, 2)
}

# Adjacency constraints
for i in range(8):
    for j in range(i + 1, 8):
        r1, c1 = positions[i]
        r2, c2 = positions[j]
        if abs(r1 - r2) <= 1 and abs(c1 - c2) <= 1:
            # If they touch (orthogonally or diagonally), No consecutives
            s.add(Abs(cells[i] - cells[j]) != 1)

if s.check() == sat:
    m = s.model()
    result = [m.evaluate(c).as_long() for c in cells]

    grid = [[' ' for _ in range(4)] for _ in range(3)]
    for idx, (r, c) in positions.items():
        grid[r][c] = str(result[idx])

    print("Solution:")
    for row in grid:
        print(' '.join(row))
else:
    print("No solution found.")


Solution:
  3 5  
7 1 8 2
  4 6  





---


# Propositional Logic

Next, we will use truth tables and z3py to prove logical entailment.

## Truth Tables

Using markdown tables and bold to show relevant rows, use the truth table method to show logical truths.


### Example

Here's an example that shows $p, p \rightarrow q \models q$.  

| p | q | p -> q |
|-------|-------|---------------------|
| **T**     | **T**     | **T**                   |
| T     | F     | F                   |
| F     | T     | T                   |
| F     | F     | T                   |

Since $q$ is true whenever $p$ and $p \rightarrow q$ are true, the expression is true.

### Questions

Prove that the following semantic entailment expressions are true by using the truth table approach.  Bold the appropriate rows **(7 point)**

 * $(p\rightarrow q)\rightarrow r, s \rightarrow \neg p, t, \neg s, t \rightarrow q \models r$
 * $\emptyset \models q \rightarrow (p \rightarrow (p \rightarrow (q \rightarrow p)))$

$(p\rightarrow q)\rightarrow r, s \rightarrow \neg p, t, \neg s, t \rightarrow q \models r$

| p | q | r | s | t | p → q | (p → q) → r | s → ¬p | ¬s | t → q | All Premises True? | r |
|---|---|---|---|---|--------|----------------|----------|-----|--------|--------------------|---|
| F | F | T | F | T |   T    |       T        |    T     |  T  |   T    | **T**             | **T** |
| F | T | T | F | T |   T    |       T        |    T     |  T  |   T    | **T**             | **T** |
| T | T | T | F | T |   T    |       T        |    T     |  T  |   T    | **T**             | **T** |


$\emptyset \models q \rightarrow (p \rightarrow (p \rightarrow (q \rightarrow p)))$

| p | q | q → (p → (p → (q → p))) |
|---|---|--------------------------|
| T | T | **T**                   |
| T | F | **T**                   |
| F | T | **T**                   |
| F | F | **T**                   |





---


 ## Logical Entailment using z3py

 Use Z3py to prove the above two entailments are true using satisfiability. See section 7.5 in the book for details. **(8 points)**


In [25]:
# answer code here

from z3 import *

#Variables
p, q, r, s, t = Bools('p q r s t')

# Premises
premises = And(
    Implies(Implies(p, q), r),
    Implies(s, Not(p)),
    t,
    Not(s),
    Implies(t, q)
)


#(p→q)→r,s→¬p,t,¬s,t→q⊨r
neg_conclusion = Not(r)
s1 = Solver()
s1.add(premises)
s1.add(neg_conclusion)


print("(p→q)→r,s→¬p,t,¬s,t→q⊨r")
if s1.check() == unsat:
    print("Holds")
else:
    print("Does Not hold: counterexample,")
    print(s1.model())


#∅⊨q→(p→(p→(q→p)))
p, q = Bools('p q')
formula = Implies(q, Implies(p, Implies(p, Implies(q, p))))
neg_formula = Not(formula)
s2 = Solver()
s2.add(neg_formula)

print("∅⊨q→(p→(p→(q→p)))")
if s2.check() == unsat:
    print("Valid")
else:
    print("Not Valid, counterexample:")
    print(s2.model())

(p→q)→r,s→¬p,t,¬s,t→q⊨r
Holds
∅⊨q→(p→(p→(q→p)))
Valid
